In [1]:
import sys
sys.path.append('../')
from run import Prepro
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import  GridSearchCV
from utils import utils_tensor as utils #import local functions

/Users/cynthiazeng/anaconda3/lib/python3.7/site-packages/tropycal/plot/plot.py:16: UserWarning: Warning: Cartopy is not installed in your python environment. Plotting functions will not work.
  warnings.warn("Warning: Cartopy is not installed in your python environment. Plotting functions will not work.")
/Users/cynthiazeng/anaconda3/lib/python3.7/site-packages/tropycal/tracks/plot.py:24: UserWarning: Warning: Cartopy is not installed in your python environment. Plotting functions will not work.
  warnings.warn("Warning: Cartopy is not installed in your python environment. Plotting functions will not work.")
/Users/cynthiazeng/anaconda3/lib/python3.7/site-packages/tropycal/tornado/dataset.py:21: UserWarning: Warning: Cartopy is not installed in your python environment. Plotting functions will not work.
  warnings.warn(warn_message)
/Users/cynthiazeng/anaconda3/lib/python3.7/site-packages/tropycal/tornado/plot.py:21: UserWarning: Warning: Cartopy is not installed in your python environm

In [ ]:
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt

In [ ]:
#process vision data to compressed vision data
#pre-specify ranks 
reduced_ranks = [3,5,3,3]

#get vision data: specificify data path here 
x_viz_train = np.load('../data/X_train_vision_1980_34_20_120_w8_at_8.npy',  allow_pickle = True).reshape(-1, steps_in , 9, 25, 25)
x_viz_test = np.load('../data/X_test_vision_1980_34_20_120_w8_at_8.npy', allow_pickle = True).reshape(-1, steps_in, 9, 25, 25)

#crop out some regions 
x_viz_train = x_viz_train[:,:,:,3:20,3:20]
x_viz_test = x_viz_test[:,:,:,3:20,3:20]

#standardize x viz
x_viz_train, x_viz_test = utils.standardize_x_viz(x_viz_train,x_viz_test)

print('compressing vision data for ranks ', reduced_ranks)
#compress vision data
x_viz_train_reduce, compress_error_train = utils.viz_reduce(x_viz_train, reduced_ranks)
x_viz_test_reduce, compress_error_test = utils.viz_reduce(x_viz_test, reduced_ranks)
print('ranks, compression error=',reduced_ranks, compress_error)

#save data: specify data out path and save 
np.save('../data/X_train_vision_comp_1980_34_20_120.npy', x_viz_train_reduce, allow_pickle=True)
np.save('../data/X_test_vision_comp_1980_34_20_120.npy', x_viz_test_reduce, allow_pickle=True)


In [5]:
# load vision and stat data 
window_size=8
predict_at = 8
reduced_ranks = [3,5,3,3]


#load x_viz: pre-processed compressed vision data 
x_viz_train = np.load('../data/X_train_vision_comp_1980_34_20_120.npy', allow_pickle=True)
x_viz_test = np.load('../data/X_test_vision_comp_1980_34_20_120.npy', allow_pickle=True)
#load x_stat: stat_data 
x_stat_train = np.load('../data/X_train_stat_1980_34_20_120_w8_at_8.npy', allow_pickle=True).reshape(-1, window_size, 30)#[:,:,:14]
x_stat_test = np.load('../data/X_test_stat_1980_34_20_120_w8_at_8.npy', allow_pickle=True).reshape(-1, window_size, 30)#[:,:,:14]
#standardize x: 
x_viz_train, x_viz_test = utils.standardize_x_viz(x_viz_train, x_viz_test)
x_stat_train, x_stat_test = utils.standardize_x_stat(x_stat_train, x_stat_test)
#concat viz and stat 
X_train_total = utils.concat_stat_viz(x_stat_train.reshape(-1, 8*30), x_viz_train.reshape(-1, 3*5*3*3))
X_test_total = utils.concat_stat_viz(x_stat_test.reshape(-1, 8*30), x_viz_test.reshape(-1, 3*5*3*3))

#load y: intensity  
tgt_intensity_train = np.load('../data/y_train_intensity_1980_34_20_120_w8_at_8.npy', allow_pickle=True)
tgt_intensity_test = np.load('../data/y_test_intensity_1980_34_20_120_w8_at_8.npy', allow_pickle=True)
#standardize y 
tgt_intensity_train, tgt_intensity_test, mean_, std_  = utils.standardize_y(tgt_intensity_train,tgt_intensity_test)

#load y: displacement 
# tgt_displacement_train = np.load('data/y_train_displacement_1980_50_20_90_w' + str(window_size) + '.npy', allow_pickle=True)
# tgt_displacement_test = np.load('data/y_test_displacement_1980_50_20_90_w' + str(window_size) + '.npy', allow_pickle=True)
#standardize y 
# tgt_dx_train,tgt_dx_test, mean_dx, std_dx  = utils.standardize_y(tgt_displacement_train[:,0],tgt_displacement_test[:,0])
# tgt_dy_train,tgt_dy_test, mean_dy, std_dy  = utils.standardize_y(tgt_displacement_train[:,1],tgt_displacement_test[:,1])

# load y: category  
# tgt_intensity_cat_train = np.load('data/y_train_intensity_cat_1980_50_20_90_w' + str(window_size) + '.npy', allow_pickle=True)
# tgt_intensity_cat_test = np.load('data/y_test_intensity_cat_1980_50_20_90_w' + str(window_size) + '.npy', allow_pickle=True)
# tgt_intensity_cat_baseline_train = np.load('data/y_train_intensity_cat_baseline_1980_50_20_90_w' + str(window_size) + '.npy',  allow_pickle = True)
# tgt_intensity_cat_baseline_test = np.load('data/y_test_intensity_cat_baseline_1980_50_20_90_w' + str(window_size) + '.npy', allow_pickle=True)


NameError: name 'window_size' is not defined

In [ ]:
tgt_displacement_train = torch.Tensor(np.load('../data/y_train_displacement_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
                                     allow_pickle=True))
tgt_displacement_test = torch.Tensor(np.load('../data/y_test_displacement_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
                                    allow_pickle=True))


In [7]:
##feature selection: skit learn has built-in feature selection function, we may not need to use iai. 
# https://machinelearningmastery.com/feature-importance-and-feature-selection-with-xgboost-in-python/ 
# here is a small example: 
from sklearn.feature_selection import SelectFromModel

X_train_sparse = X_train.copy()

# fit model on training data
#model = XGBRegressor(max_depth=5, n_estimators=100)
#model.fit(X_train, tgt_intensity_train)

# select features using threshold


xgb2 = XGBRegressor(max_depth=8, n_estimators=140, learning_rate = 0.07, subsample = 0.7, min_child_weight = 5)
xgb2.fit(X_train_total, tgt_intensity_train)
select = SelectFromModel(xgb2, threshold="mean", prefit=True)
X_train_sparse = select.transform(X_train_total)
X_test_sparse = select.transform(X_test_total)
print("MAE intensity: ", mean_absolute_error(np.array(tgt_intensity_test)*std_+mean_, np.array(xgb2.predict(X_test_total))*std_+mean_))


xgb2.fit(X_train_sparse, tgt_intensity_train)
print("MAE intensity: ", mean_absolute_error(np.array(tgt_intensity_test)*std_+mean_, np.array(xgb2.predict(X_test_sparse))*std_+mean_))

In [ ]:
model_bytearray = xgb2.save_raw()[4:]
def myfun(self=None):
    return model_bytearray

xgb2.save_raw = myfun

In [ ]:
X_train = np.load('../data/X_train_stat_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
            allow_pickle=True)
X_test = np.load('../data/X_test_stat_1980_34_20_120_w' + str(window_size) + '_at_' + str(predict_at) + '.npy',
            allow_pickle=True)


names = ['LAT', 'LON', 'WMO_WIND', 'WMO_PRES', 'DIST2LAND',
         'STORM_SPEED', 'cat_cos_day', 'cat_sign_day', 'COS_STORM_DIR', 'SIN_STORM_DIR',
         'COS_LAT', 'SIN_LAT', 'COS_LON', 'SIN_LON', 'cat_storm_category', 'cat_basin_AN',
         'cat_basin_EP', 'cat_basin_NI', 'cat_basin_SA',
         'cat_basin_SI', 'cat_basin_SP', 'cat_basin_WP', 'cat_nature_DS', 'cat_nature_ET',
         'cat_nature_MX', 'cat_nature_NR', 'cat_nature_SS', 'cat_nature_TS',
         'STORM_DISPLACEMENT_X', 'STORM_DISPLACEMENT_Y']

names_all = names * window_size

for i in range(len(names_all)):
    names_all[i] += '_' + str(i // 30)

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
X_train.columns = names_all
X_test.columns = names_all

cols = [c for c in X_train.columns if c.lower()[-2:] == '_0' or c.lower()[:3] != 'cat']

X_train = X_train[cols]
X_test = X_test[cols]

X_test_baseline = pd.DataFrame(np.load('../data/X_test_stat_1980_34_20_120_withforecast_2661_w' + str(window_size) + '_at_' + str(predict_at) + '.npy', allow_pickle=True))

X_test_dates = pd.DataFrame(np.load('../data/X_test_stat_with_dates_columns_1980_w' + str(window_size) + '_at_' + str(predict_at) + '.npy', allow_pickle=True)[:,:4])

X_test_dates.columns = ['YEAR', 'MONTH', 'DAY', 'HOUR']

names_baselines = [#'SID',
         'LAT', 'LON', 'WMO_WIND', 'WMO_PRES', 'DIST2LAND',
         'STORM_SPEED', 'cat_cos_day', 'cat_sign_day', 'COS_STORM_DIR', 'SIN_STORM_DIR',
         'COS_LAT', 'SIN_LAT', 'COS_LON', 'SIN_LON', 'cat_storm_category',
         'EMXI_24_lat', 'EMXI_24_lon', 'EMXI_24_vmax', 'EMXI_24_mslp',
         'SHIP_24_lat', 'SHIP_24_lon', 'SHIP_24_vmax', 'SHIP_24_mslp',
         'AP01_24_lat', 'AP01_24_lon', 'AP01_24_vmax', 'AP01_24_mslp',
         'CMC_24_lat', 'CMC_24_lon', 'CMC_24_vmax', 'CMC_24_mslp',
         'NAM_24_lat', 'NAM_24_lon', 'NAM_24_vmax', 'NAM_24_mslp',
         'HWRF_24_lat', 'HWRF_24_lon', 'HWRF_24_vmax', 'HWRF_24_mslp',
         'cat_basin_AN', 'cat_basin_EP', 'cat_basin_NI', 'cat_basin_SA',
         'cat_basin_SI', 'cat_basin_SP', 'cat_basin_WP',
         #'cat_nature_DS', 'cat_nature_ET',
         #'cat_nature_MX', 'cat_nature_NR', 'cat_nature_SS', 'cat_nature_TS',
         'STORM_DISPLACEMENT_X', 'STORM_DISPLACEMENT_Y']

names_all_baselines = names_baselines * 8#args.window_size

for i in range(len(names_all_baselines)):
    names_all_baselines[i] += '_' + str(i // 48)

X_test_baseline.columns = names_all_baselines

X_test_baseline = pd.concat([X_test_baseline, X_test_dates], axis = 1)



In [ ]:
train_xgb_intensity(forecast = 'SHIP', basin = 'AN', max_depth=8, n_estimators = 140, learning_rate = 0.07, subsample = 0.7, min_child_weight = 5, forecast2 = 'HWRF')

In [ ]:
##grid search 
grid = GridSearchCV(estimator = XGBRegressor(learning_rate = 0.15, n_estimators=140, max_depth=5,
 min_child_weight=1, subsample=0.8, seed=1),
 param_grid = params, n_jobs=4, scoring = 'neg_mean_absolute_error')

grid.fit(X_train_sparse, tgt_intensity_train)

params = {
 'min_child_weight':range(1,6,2),
 'n_estimators':[100, 140],
 'subsample':[0.6,0.8,1],
 'learning_rate':[0.1, 0.15, 0.2]
}

grid.grid_scores_, grid.best_params_, grid.best_score_